In [3]:
!pip install jupyter-dash

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

In [3]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

OSError: Unable to communicate with the jupyter_dash notebook or JupyterLab 
extension required to infer Jupyter configuration.

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
marks_range = {i: str(i) for i in range(0, 12500, 2500)}

launch_sites = spacex_df['Launch Site'].unique().tolist()
launch_sites_labels = [{'label': 'All Sites', 'value': 'ALL'}]
launch_sites_options = {}
for i, site in enumerate(launch_sites):
    launch_sites_labels.append({'label': site, 'value': f'SITE{i}'})
    launch_sites_options[f'SITE{i}'] = site

In [ ]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=launch_sites_labels,
                                             value='ALL',
                                             placeholder='Select a Launch Site',
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks=marks_range,
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))
def get_graph_pie(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df[spacex_df['class'] == 1]
        filtered_df = filtered_df.groupby('Launch Site')['class'].count().reset_index()
        fig = px.pie(filtered_df, values='class',
        names='Launch Site',
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == launch_sites_options[entered_site]]
        filtered_df = filtered_df.groupby('class')['Launch Site'].count().reset_index()
        fig = px.pie(filtered_df, values='Launch Site',
        names='class',
        title=f'Total Success Launches for Site {launch_sites_options[entered_site]}')
        return fig


In [ ]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value'))
def get_graph_scatter(entered_site, entered_range):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(entered_range[0], entered_range[1], inclusive=True)]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)', y='class', color="Booster Version Category",
                         title="Correlation between Payload and Success for all Sites")
        return fig
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == launch_sites_options[entered_site]]
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)', y='class', color="Booster Version Category",
                         title=f"Correlation between Payload and Success for Site {launch_sites_options[entered_site]}")
        return fig

In [ ]:
app = JupyterDash(__name__)

In [ ]:
if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost")